In [34]:
import re
import numpy as np
import pandas as pd 
import os
import sys
import itertools
import io
import datetime

In [35]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

### Establishes General Path

In [36]:
##local_path = "/your/path/here"
local_path = "/users/thudson/Documents/Dropbox/chicago-police-data/02-column_std"

In [37]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/output')

['incomplete',
 'TRRs',
 'complaints-cpd-2016-dec',
 'complaints-cpd-2016-jun',
 'complaints-cpd-2016-nov',
 'complaints-cpd-2016-oct',
 'complaints-ipra-2016-apr',
 'complaints-merged-2015',
 'shootings-cpd-feb2016',
 'shootings-ipra-may2016']

In [38]:
in_path = path + '/output/complaints-cpd-2016-nov/'
out_path = local_path + '/output/complaints-cpd-2016-nov/'

In [6]:
os.mkdir(out_path)

In [39]:
##doc_path = '/Data/chicago-police-data/02-column_std'
##column_dict = db.download_file(doc_path+'/doc/','Column_Dictionary.csv')

doc_path = local_path
column_dict = pd.read_csv(doc_path+'/doc/'+'Column_Dictionary.csv').iloc[:,0:2]
column_dict.head()

,Orig Column Name,New Name
0,Victim_Age,Civilian.Victim.Age
1,Victim_Gender,Civilian.Victim.Gender
2,Victim_Race,Civilian.Victim.Race_Code
3,Witness_Age,Civilian.Witness.Age
4,Witness_Gender,Civilian.Witness.Gender


In [40]:
## SOMETHING TO PUT IN UTILS? HOW WOULD THE SPECIFIED COLUMN_DICT WORK?

def column_mapper(df,
                  column_dict = column_dict):
    current_columns = pd.DataFrame(df.columns.tolist(),columns=['Orig Column Name'])
    new_columns = current_columns.merge(column_dict,how='left',on='Orig Column Name')['New Name'].tolist() 
    if current_columns.shape[0] == len(new_columns):
        return new_columns
    else:
        print("Join failed, more columns returned than expected")

## Nov 2016 Data

### Report 1

In [9]:
in_path_nov_2016_report = in_path
out_path_nov_2016_report = out_path


files = db.list_files(in_path_nov_2016_report)
files

['p046957_-_report_1_-_all_complaints_in_time_frame.csv',
 'p046957_-_report_1_-_all_complaints_in_time_frame.xlsx',
 'p046957_-_report_1_-_all_complaints_in_time_frame_metadata.csv',
 'p046957_-_report_2_-_identified_accused.csv',
 'p046957_-_report_2_-_identified_accused.xlsx',
 'p046957_-_report_2_-_identified_accused_metadata.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.xlsx',
 'p046957_-_report_3_-_police_officer_witness_data_xi_metadata.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.xlsx',
 'p046957_-_report_5_-_complainant_(reporting_party)_data_metadata.csv']

In [10]:
file = files[0]
df = db.download_file(in_path_nov_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-cpd-2016-nov/p046957_-_report_1_-_all_complaints_in_time_frame.csv


/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


['CRID',
 'Beat',
 'Location_Code',
 'Location_Value',
 'Address_of_Incident',
 'City',
 'State',
 'Zip',
 'Incident_Date',
 'Complaint_Date',
 'Closed_Date',
 'Investigator_Full_Name',
 'Investigator_Assignment',
 'Investigator_Rank',
 'Investigator_Star',
 'Investigator_Appt_Date',
 'FOIA_Request_Number']

In [11]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Incident.Location.Beat',
 'Incident.Location.Location_Code',
 'Incident.Location.Location_Value',
 'Incident.Location.Address',
 'Incident.Location.City',
 'Incident.Location.State',
 'Incident.Location.Zip',
 'Incident.Time.Date',
 'Complaint.Date',
 'Complaint.Current.Closed_date',
 'Investigator.Full_Name',
 'Investigator.Assignment',
 'Investigator.Rank',
 'Investigator.Star',
 'Investigator.Appt_Date',
 'FOIA_Request_Number']

In [12]:
df.to_csv(out_path_nov_2016_report+file,index=False)

### Report 2

In [41]:
in_path_nov_2016_report = in_path
out_path_nov_2016_report = out_path


files = db.list_files(in_path_nov_2016_report)
files

['p046957_-_report_1_-_all_complaints_in_time_frame.csv',
 'p046957_-_report_1_-_all_complaints_in_time_frame.xlsx',
 'p046957_-_report_1_-_all_complaints_in_time_frame_metadata.csv',
 'p046957_-_report_2_-_identified_accused.csv',
 'p046957_-_report_2_-_identified_accused.xlsx',
 'p046957_-_report_2_-_identified_accused_metadata.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.xlsx',
 'p046957_-_report_3_-_police_officer_witness_data_xi_metadata.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.xlsx',
 'p046957_-_report_5_-_complainant_(reporting_party)_data_metadata.csv',
 'p046957_-_report_4_-_victim_data_metadata.csv',
 'p046957_-_report_4_-_victim_data.csv',
 'p046957_-_report_4_-_victim_data.xlsx']

In [42]:
file = files[3]
df = db.download_file(in_path_nov_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-cpd-2016-nov/p046957_-_report_2_-_identified_accused.csv


/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


['CRID',
 'Accused',
 'Accused_Birth_Year',
 'Accused_Gender',
 'Accused_Race_Code',
 'Date_of_Appt',
 'Current_Unit',
 'Current_Rank',
 'Star',
 'Complaint_Category',
 'Orig_Finding',
 'Orig_Recommended_Discipline',
 'Final_Finding',
 'Final_Recommended_Discipline',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [44]:
df.shape

(125581, 16)

In [45]:
def Complaint_Category_Splitter(string):
    new_string = str(string).split('-',1)
    if len(new_string)==1:
        new_string= ['nan','nan']
    return new_string

split_df = pd.DataFrame(df.Complaint_Category.apply(Complaint_Category_Splitter).tolist(),
                        columns = ['Complaint_Category','Complaint_Description'])
    

df = df.drop("Complaint_Category",axis=1)
df1 = df.merge(split_df,how='left',left_index=True,right_index=True)
df1.shape

(125581, 17)

In [47]:
df = df1

In [48]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Officer.Accused.Full_Name',
 'Officer.Accused.Birth_Year',
 'Officer.Accused.Gender',
 'Officer.Accused.Race_Code',
 'Officer.Accused.Appointed_Date',
 'Officer.Accused.Current_Unit',
 'Officer.Accused.Current_Rank',
 'Officer.Accused.Star',
 'Complaint.Initial.Finding',
 'Complaint.Initial.Recommended_Discipline',
 'Complaint.Final.Finding',
 'Complaint.Final.Recommended_Discipline',
 'FOIA_Request_Number',
 'Report_Produced_Date',
 'Complaint.Current.Category',
 'Complaint.Current.Description']

In [49]:
df.to_csv(out_path_nov_2016_report+file,index=False)

### Report 3

In [30]:
in_path_nov_2016_report = in_path
out_path_nov_2016_report = out_path


files = db.list_files(in_path_nov_2016_report)
files

['p046957_-_report_1_-_all_complaints_in_time_frame.csv',
 'p046957_-_report_1_-_all_complaints_in_time_frame.xlsx',
 'p046957_-_report_1_-_all_complaints_in_time_frame_metadata.csv',
 'p046957_-_report_2_-_identified_accused.csv',
 'p046957_-_report_2_-_identified_accused.xlsx',
 'p046957_-_report_2_-_identified_accused_metadata.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.xlsx',
 'p046957_-_report_3_-_police_officer_witness_data_xi_metadata.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.xlsx',
 'p046957_-_report_5_-_complainant_(reporting_party)_data_metadata.csv',
 'p046957_-_report_4_-_victim_data_metadata.csv',
 'p046957_-_report_4_-_victim_data.csv',
 'p046957_-_report_4_-_victim_data.xlsx']

In [31]:
file = files[6]
df = db.download_file(in_path_nov_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-cpd-2016-nov/p046957_-_report_3_-_police_officer_witness_data_xi.csv


['CRID',
 'Officer_Witness',
 'Officer_Witness_Gender',
 'Officer_Witness_Race',
 'Officer_Witness_Star',
 'Officer_Witness_Birth_Year',
 'Officer_Witness_Date_Appointed',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [32]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Officer.Witness.Full_Name',
 'Officer.Witness.Gender',
 'Officer.Witness.Race_Code',
 'Officer.Witness.Star',
 'Officer.Witness.Birth_Year',
 'Officer.Witness.Appointed_Date',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [33]:
df.to_csv(out_path_nov_2016_report+file,index=False)

### Report 4

In [20]:
in_path_nov_2016_report = in_path
out_path_nov_2016_report = out_path


files = db.list_files(in_path_nov_2016_report)
files

['p046957_-_report_1_-_all_complaints_in_time_frame.csv',
 'p046957_-_report_1_-_all_complaints_in_time_frame.xlsx',
 'p046957_-_report_1_-_all_complaints_in_time_frame_metadata.csv',
 'p046957_-_report_2_-_identified_accused.csv',
 'p046957_-_report_2_-_identified_accused.xlsx',
 'p046957_-_report_2_-_identified_accused_metadata.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.xlsx',
 'p046957_-_report_3_-_police_officer_witness_data_xi_metadata.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.xlsx',
 'p046957_-_report_5_-_complainant_(reporting_party)_data_metadata.csv',
 'p046957_-_report_4_-_victim_data_metadata.csv',
 'p046957_-_report_4_-_victim_data.csv',
 'p046957_-_report_4_-_victim_data.xlsx']

In [21]:
file = files[9]
df = db.download_file(in_path_nov_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-cpd-2016-nov/p046957_-_report_5_-_complainant_(reporting_party)_data.csv


['CRID',
 'Witness_Gender',
 'Witness_Age',
 'Witness_Race',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [22]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Civilian.Witness.Gender',
 'Civilian.Witness.Age',
 'Civilian.Witness.Race_Code',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [23]:
df.to_csv(out_path_nov_2016_report+file,index=False)

### Report 5

In [24]:
in_path_nov_2016_report = in_path
out_path_nov_2016_report = out_path


files = db.list_files(in_path_nov_2016_report)
files

['p046957_-_report_1_-_all_complaints_in_time_frame.csv',
 'p046957_-_report_1_-_all_complaints_in_time_frame.xlsx',
 'p046957_-_report_1_-_all_complaints_in_time_frame_metadata.csv',
 'p046957_-_report_2_-_identified_accused.csv',
 'p046957_-_report_2_-_identified_accused.xlsx',
 'p046957_-_report_2_-_identified_accused_metadata.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.xlsx',
 'p046957_-_report_3_-_police_officer_witness_data_xi_metadata.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.xlsx',
 'p046957_-_report_5_-_complainant_(reporting_party)_data_metadata.csv',
 'p046957_-_report_4_-_victim_data_metadata.csv',
 'p046957_-_report_4_-_victim_data.csv',
 'p046957_-_report_4_-_victim_data.xlsx']

In [27]:
file = files[13]
df = db.download_file(in_path_nov_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-cpd-2016-nov/p046957_-_report_4_-_victim_data.csv


['CRID',
 'Victim_Gender',
 'Victim_Age',
 'Victim_Race',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [28]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Civilian.Victim.Gender',
 'Civilian.Victim.Age',
 'Civilian.Victim.Race_Code',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [29]:
df.to_csv(out_path_nov_2016_report+file,index=False)